In [2]:
import pandas as pd
pd.set_option("display.max_row", 1000)
pd.set_option("display.max_column", 100)
import warnings
warnings.filterwarnings('ignore')

path = "https://raw.githubusercontent.com/Soyoung-Yoon/data_01/main/"

**18-1) 수도권 지역 매출 분석**

다음은 지역별 제품 매출 데이터를 담고 있는 데이터셋이다.
데이터는 region, product, sales, date 컬럼으로 구성되어 있으며,
sales는 각 제품의 일별 매출액을 나타낸다.

다음에 따라 데이터를 처리하시오.
- 1) 각 지역(region)별 평균 매출액(sales)을 계산한다.
- 2) 각 지역별로 해당 지역의 평균 매출액보다 큰 매출액을 가진 데이터의 개수를 구한다.
- 3) 위에서 구한 데이터의 개수가 가장 큰 지역의 제품 'A' 또는 'B'에 대한 sales 총합을 구한다.

In [ ]:
df = pd.read_csv(path + "sales_data02.csv")
# print(df.head(3))

# 1. 각 지역(region)별 평균 매출액(sales)을 계산한다.
s = df.groupby('region')['sales'].transform('mean')
# print(s)

# 2. 각 지역별로 해당 지역의 평균 매출액보다 큰 매출액을 가진 데이터의 개수를 구한다.
df2 = df[df['sales'] > s].copy()
max_region = df2['region'].value_counts().index[0]
# print(max_region)

#  3. 위에서 구한 데이터의 개수가 가장 큰 지역의 제품 'A' 또는 'B'에 대한 sales 총합을 구한다.
cond = (df['region'] == max_region) & (df['product'].isin(['A','B']))
result = df.loc[cond, 'sales'].sum()
# print(result) # 34781 --> 10120

10120


**18-2) 연도별 질병 사망률 분석**

2014년부터 2024년까지의 주요 10대 질병에 대한 사망자수 및 발생건수를 기록한 데이터를 사용하여 사망률을 분석한다.

다음 조건에 따라 분석을 수행하시오.
- 1) 각 연도별로 각 질병에 대해 생존률을 계산한다.
> 생존률 = 1 - (사망자수 / 발생건수)
- 2) 연도별로 생존률이 가장 높은 질병명을 1개씩 선택한다.
- 3) 선택된 연도별, 생존률이 가장 높은 질병들에 대해, 생존자수 총합을 정수형으로 출력한다.
> 생존자수 = 발생건수 - 사망자수

In [ ]:
df = pd.read_csv(path + "disease_data.csv")
# print(df.head(3))

# 1. 각 연도별로 각 질병에 대해 생존률을 계산한다.
# 생존률 = 1 - (사망자수 / 발생건수)
df = df.set_index('연도')
df1 = df[df['구분']=='사망자수'].drop(columns='구분') # 연도, 질병별 사망자수
df2 = df[df['구분']=='발생건수'].drop(columns='구분')
rate = 1 - (df1 / df2)
# print(rate)

# 2. 연도별로 생존률이 가장 높은 질병명을 1개씩 선택한다.
s = rate.idxmax(axis=1)

# 3. 선택된 연도별, 생존률이 가장 높은 질병들에 대해, 생존자수 총합을 정수형으로 출력한다.
#    생존자수 = 발생건수 - 사망자수
df3 = df2 - df1
temp = [df3.loc[year, name] for year, name in zip(s.index, s.values)]
result = int(sum(temp))
print(result) # 27021



27021


**18-3) 가구별 에너지 소비 격차 분석**

가구의 연도별 에너지 사용량 데이터를 사용하여, 지역별 에너지 사용 패턴을 분석하여 에너지 소비 불균형 여부를 파악하고자 한다.

다음 조건에 따라 분석을 수행하시오.
- 1) Electricity와 Gas 사용량의 합을 '총에너지사용량'으로 계산하시오.
- 2) 지역별로 연도(Year) 및 계절(Season)별 '총에너지사용량' 합계를 구하시오.
- 3) 위의 결과를 사용해 각 지역(Region)별 에너지 사용량 평균과 중앙값의 절댓값 차이가 가장 큰 지역 이름을 문자열로 출력하시오.

In [ ]:
df = pd.read_csv(path + "energy_data.csv")
print(df.head(3))

# 1. Electricity와 Gas 사용량의 합을 '총에너지사용량'으로 계산하시오.
df['Energy'] = df['Electricity'] + df['Gas']
# print(df.head(3))

# 2. 지역별로 연도(Year) 및 계절(Season)별 '총에너지사용량' 합계를 구하시오.
df2 = df.groupby(['Region','Year','Season'])['Energy'].sum().reset_index()
# print(df2.head(3))

# 3. 위의 결과를 사용해 각 지역(Region)별 에너지 사용량 평균과 중앙값의 절댓값 차이가 가장 큰 지역 이름을 문자열로 출력하시오.
df3 = df2.groupby('Region')['Energy'].agg(['mean', 'median'])
result = (df3['mean'] - df3['median']).abs().idxmax()
# print(result) # 서울


   Year  Season Region  Electricity  Gas
0  2018  Spring     서울          313  321
1  2018  Spring     서울          877  363
2  2018  Spring     서울          254  557
서울


**18-4) 차량 종류별 비율 분석**

도로별, 차량유형별 통행량이 포함된 데이터를 사용하여 결측치를 해결하고, 통행량 분석을 한다.

다음 조건에 따라 분석을 수행하시오.
- 1) Count 컬럼의 결측치는 Date별, Vehicle_Type별 평균값으로 대체하시오.
- 2) 1)의 결과에 대해, 일요일에 대한 데이터를 A, 일요일이 아닌 데이터를 B라고 한다.
- 3) Road_Code별 통행량 평균을 A, B 각각에 대해  구한 뒤, B - A의 절대값을 구한다.
- 4) 3)에서 구한 값 중 가장 큰 값에 대한 Road_Code를 찾아 출력하시오.

In [ ]:
df = pd.read_csv(path + "traffic_data02.csv")
# print(df.head(3))

# 1. Count 컬럼의 결측치는 Date별, Vehicle_Type별 평균값으로 대체하시오.
# print(df.isna().sum().to_frame().T)
s = df.groupby(['Date','Vehicle_Type'])['Count'].transform('mean')
df['Count'] = df['Count'].fillna(s)
# print(df.isna().sum().to_frame().T)

# 2. 1)의 결과에 대해, 일요일에 대한 데이터를 A, 일요일이 아닌 데이터를 B라고 한다.
df['Date'] = pd.to_datetime(df['Date'])
df['Daynm'] = df['Date'].dt.day_name('ko_KR')
# print(df.head(3))
A = df[df['Daynm'] == '일요일'].copy()
B = df[df['Daynm'] != '일요일'].copy()
# print(A.shape, B.shape) # (40, 5) (208, 5)

# 3. Road_Code별 통행량 평균을 A, B 각각에 대해 구한 뒤, B - A의 절대값을 구한다.
A_mean = A.groupby('Road_Code')['Count'].mean().reset_index(name='A_mean')
B_mean = B.groupby('Road_Code')['Count'].mean().reset_index(name='B_mean')
diff = pd.merge(B_mean, A_mean, "left", "Road_Code")
diff['mean_diff'] = abs(diff['B_mean'] - diff['A_mean'])

# 4. 3)에서 구한 값 중 가장 큰 값에 대한 Road_Code를 찾아 출력하시오.
max_diff = diff['mean_diff'].max()
result = diff.loc[diff['mean_diff'] == max_diff, 'Road_Code'].values[0]
print(result) # 204


204


**18-5) 이상치 발생 빈도 분석**

이벤트별 이상치의 개수를 분석하고자 한다.

다음 조건에 따라 분석을 수행하시오.
- 1) start_datetime을 기준으로, 1일부터 15일까지의 날짜에 해당하는 데이터만 대상으로 분석한다.
- 2) 이벤트(event) 종류 별로 다음 기준에 따라 value 컬럼의 이상치 개수를 구해, 그 합을 출력하시오.
> 이상치 = 값이 Q1 - 1.5 × IQR 미만이거나 Q3 + 1.5 × IQR 초과하는 데이터
- 단, IQR = Q3 - Q1 (사분위 범위)


In [ ]:
df = pd.read_csv(path + "event_log_01b_outlier.csv")
# print(df.head(3))

# 1. start_datetime을 기준으로, 1일부터 15일까지의 날짜에 해당하는 데이터만 대상으로 분석한다.
s = df['start_datetime'].astype('datetime64[ns]').dt.day
df = df[(s >= 1) & (s <= 15)].copy()

# 2. 이벤트(event) 종류 별로 다음 기준에 따라 value 컬럼의 이상치 개수를 구해, 그 합을 출력하시오.
#    이상치 = 값이 Q1 - 1.5 × IQR 미만이거나 Q3 + 1.5 × IQR 초과하는 데이터
#    단, IQR = Q3 - Q1 (사분위 범위)
# Q1, Q3 = df['value'].quantile([0.25, 0.75])
Q1 = df.groupby('event')['value'].transform(lambda x: x.quantile(0.25))
Q3 = df.groupby('event')['value'].transform(lambda x: x.quantile(0.75))
IQR = Q3 - Q1
lower = Q1 - 1.5 * IQR
upper = Q3 + 1.5 * IQR
temp = df[(df['value'] < lower) | (df['value'] > upper)]
result = len(temp)
print(result) # 56 --> 58

58
